In [126]:
import pandas as pd 

In [127]:
df = pd.read_csv('spotify_millsongdata.csv')

In [128]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [129]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [130]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [131]:
df.head()

,artist,song,text
0,Yeah Yeah Yeahs,Miles Away,As the cars go by \r\nUnder the sun like an e...
1,Alice Cooper,Refrigerator Heaven,"I'm freezing, I'm frozen, I'm icicle blue \r\..."
2,King Diamond,I'm Not A Stranger,I've been watching Lucy all week \r\nI've bee...
3,Whitesnake,Ready To Rock,I'm getting ready to rock \r\nI'm getting rea...
4,Genesis,Blood On The Rooftops,"Dark and grey, an English film, the Wednesday ..."


In [132]:
df = df.sample(5000)

In [133]:
df.shape

(5000, 3)

Text preprocessing

In [134]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', '').replace(r'\n', ' ', regex=True)

In [135]:
df.head()

,artist,song,text
837,Who,Sea And Sand,here by the sea and sand \r nothing ever goes...
4031,Eric Clapton,Anything For Your Love,"god's got a plan, he's making it clear \r swe..."
1793,Chris Rea,The Road To Hell (Part 2),well i'm standing by the river \r but the wat...
393,Grand Funk Railroad,Gotta Find Me A Better Day,"i been gettin' lonely, too much these days. \..."
4836,Indiana Bible College,Psalm 23,"chorus \r the lord is my shepherd, and i \r ..."


Converting text in token 

In [136]:
import nltk
from nltk.stem.porter import PorterStemmer

In [137]:
stem_1 = PorterStemmer()

In [138]:
def token(txt):
    token = nltk.word_tokenize(txt)
    s = [stem_1.stem(w)for w in token]
    return " ". join(s)

In [139]:
df['text'].apply(lambda x: token(x))

837     here by the sea and sand noth ever goe as plan...
4031    god 's got a plan , he 's make it clear sweet ...
1793    well i 'm stand by the river but the water doe...
393     i been gettin ' lone , too much these day . i ...
4836    choru the lord is my shepherd , and i i shall ...
                              ...                        
3110    i 'm just a man.i breath and i bleed ye i got ...
4150    see the staaaaaaar they ai n't faaaaaaar you c...
2851    i want to vanish thi is my fondest wish to go ...
4829    we are boy scout , trust in our honour . loyal...
4744    oh no oh no oh no you never make me stay so ta...
Name: text, Length: 5000, dtype: object

In [140]:
#token("")

Converting text and token into vectors 

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
#importing metric 
from sklearn.metrics.pairwise import cosine_similarity 

In [142]:
tfid =TfidfVectorizer(analyzer='word', stop_words='english')

In [143]:
matrix=  tfid.fit_transform(df['text'])

In [144]:
similar= cosine_similarity(matrix)

In [145]:
similar[0]

array([1.        , 0.06333901, 0.04144369, ..., 0.00307282, 0.00917518,
       0.01726931])

Creating the recomendation system 

In [146]:
df[df['song']== 'Bang']

,artist,song,text


In [150]:
def recommender(song_name):
    idx = df[df['song']== song_name].index[0]
    distance= sorted(list(enumerate(similar[idx])), reverse=True, key = lambda x:x[1])
    song= []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [151]:
recommender('Sea And Sand')

['Back In White', 'False Alarm', 'Amlapura', 'Hey Girl']

In [152]:
import pickle

In [154]:
pickle.dump(similar,open("similarity","wb"))

In [156]:
pickle.dump(df,open("df", "wb"))